In [1]:
import pandas as pd
import sqlite3


In [3]:
import pandas as pd
import sqlite3
import glob

# Correct folder path (ensure it does NOT end with `\` to avoid double slashes)
folder_path = r"C:\Users\doris\Desktop\Data management\archive\dataset"

# Get all CSV file paths in the folder
csv_files = glob.glob(folder_path + "/*.csv")  # Uses `/*.csv` to find all CSVs

# Check if files are found
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {folder_path}")

# Connect to SQLite database (in-memory)
conn = sqlite3.connect(":memory:")

# Loop through each CSV file and load into SQLite
for file in csv_files:
    file_name = file.split("\\")[-1].replace(".csv", "")  # Extracts file name (without extension)
    clean_name = file_name.replace(" ", "_")  # Replaces spaces with underscores to prevent SQL errors

    print(f"Loading: {clean_name}")  # Print which file is being loaded
    
    # Read CSV with error handling
    try:
        df = pd.read_csv(file, low_memory=False)
    except Exception as e:
        print(f"❌ Error loading {file_name}: {e}")
        continue  # Skip this file and continue with the next one

    # Store DataFrame in SQLite with the cleaned table name
    df.to_sql(clean_name, conn, index=False, if_exists="replace")

print("✅ All CSV files successfully loaded!")

# Query an example table (Change `table_name` to any loaded table)
query = f"SELECT * FROM Amazon_Sale_Report LIMIT 5;"  # Uses the last loaded table as default
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)



Loading: Amazon_Sale_Report
Loading: International_sale_Report
Loading: Sale_Report
✅ All CSV files successfully loaded!


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,None,0,Easy Ship,None
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,0,Easy Ship,None
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,1,None,None
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,None,0,Easy Ship,None
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,None,0,None,None


In [12]:
# Connect 3 tables together 
query = """
SELECT asr.*, sr.*, isr.*
FROM Amazon_Sale_Report asr 
LEFT JOIN Sale_Report sr ON asr.sku = sr.[SKU Code] 
LEFT JOIN International_sale_Report isr ON asr.sku = isr.sku;
"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)



,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,index,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,7911.0,01-29-22,Jan-22,MULBERRIES BOUTIQUE,SET389,SET389-KR-NP-S,S,1.00,1000.00,1000.00
1,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,13607.0,01-29-22,Jan-22,MULBERRIES BOUTIQUE,SET389,SET389-KR-NP-S,S,1.00,1000.00,1000.00
2,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,27846.0,MULBERRIES BOUTIQUE,01-29-22,Jan-22,SET389,SET389-KR-NP-S,1.00,1000.00,1000.00,60.00
3,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,33711.0,MULBERRIES BOUTIQUE,01-29-22,Jan-22,SET389,SET389-KR-NP-S,1.00,1000.00,1000.00,60.00
4,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,NaN,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828633,128973,402-6184140-0545956,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0012,J0012-SKD-XS,Set,...,33678.0,MULBERRIES BOUTIQUE,01-29-22,Jan-22,J0012,J0012-SKD-XS,1.00,1387.50,1388.00,119.00
828634,128974,408-7436540-8728312,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0003,J0003-SET-S,Set,...,7990.0,01-29-22,Jan-22,MULBERRIES BOUTIQUE,J0003,J0003-SET-S,S,1.00,737.50,738.00
828635,128974,408-7436540-8728312,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0003,J0003-SET-S,Set,...,13686.0,01-29-22,Jan-22,MULBERRIES BOUTIQUE,J0003,J0003-SET-S,S,1.00,737.50,738.00
828636,128974,408-7436540-8728312,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0003,J0003-SET-S,Set,...,27930.0,MULBERRIES BOUTIQUE,01-29-22,Jan-22,J0003,J0003-SET-S,1.00,737.50,738.00,15.00


In [13]:
# Close connection
conn.close()
